In [1]:
# CÓDIGO A EJECUTAR EN LA CELDA 1 (SOFASCORE)

import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from sklearn.ensemble import RandomForestClassifier

# URL principal de la Liga Chilena (Temporada 2025, ID 71131)
URL_LIGA_SOFASCORE = 'https://www.sofascore.com/es/torneo/futbol/chile/primera-division/11653#id:71131'
# URL base de la API de estadísticas individuales
BASE_URL_ESTADISTICAS_API = 'https://www.sofascore.com/api/v1/event/'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'Accept': 'application/json, text/plain, */*', 
}

print("Configuración y librerías de Selenium cargadas.")

Configuración y librerías de Selenium cargadas.


In [2]:
# CÓDIGO A EJECUTAR EN LA CELDA 2

def obtener_ids_partidos_sofascore(url_liga_general):
    """
    Usa Selenium para cargar la página de resultados de SofaScore y extraer los IDs de los partidos 
    de los enlaces visibles.
    """
    print("Iniciando Selenium para cargar la tabla de resultados...")
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options) 
    
    driver.get(url_liga_general)
    driver.implicitly_wait(10)
    
    # 1. Esperar a que los enlaces de partido se carguen (usaremos el patrón 'data-id')
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-id]'))
        )
        print("Tabla principal de partidos cargada.")
    except Exception:
        print("Error: No se encontraron enlaces de partido en la página.")
        driver.quit()
        return []
    
    # 2. Obtener el HTML y extraer los IDs
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit() 
    
    # Buscamos todos los elementos 'a' que contienen el ID del evento (data-id)
    eventos = soup.find_all('a', attrs={'data-id': True})
    
    lista_ids = []
    for evento in eventos:
        # El ID real es el que está en el atributo 'data-id'
        partido_id = evento['data-id']
        # El slug está en el atributo 'href'
        slug = evento['href']
        
        lista_ids.append({'ID_Partido': partido_id, 'Slug_URL': slug})
            
    print(f"Se recolectaron {len(lista_ids)} URLs/IDs de partidos.")
    return lista_ids

# Ejecución de la Fase 1
datos_partidos_base = obtener_ids_partidos_sofascore(URL_LIGA_SOFASCORE)

Iniciando Selenium para cargar la tabla de resultados...
Error: No se encontraron enlaces de partido en la página.


In [ ]:
# CÓDIGO A EJECUTAR EN LA CELDA 3

def obtener_estadisticas_avanzadas_sofascore(row):
    """
    Intenta acceder a la API de estadísticas individuales usando el ID recolectado.
    """
    event_id = row['ID_Partido']
    # Usamos la API de estadísticas que requiere el ID
    api_url = f"{BASE_URL_ESTADISTICAS_API}{event_id}/statistics" 
    
    stats = {'xG_Local': None, 'Posesion_Local': None, 'Tiros_Arco_Local': None} 
    
    try:
        # Reutilizamos los HEADERS simples y la librería requests (la única forma de leer JSON)
        response = requests.get(api_url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        
        # --- Lógica de Extracción de Estadísticas (Estructura SofaScore) ---
        stat_groups = json_data.get('statistics', [{}])[0].get('groups', [])
        
        for group in stat_groups:
            for stat in group.get('statisticsItems', []):
                name = stat.get('name')
                
                if name == 'Expected goals':
                    stats['xG_Local'] = stat.get('homeValue')
                elif name == 'Ball possession':
                    stats['Posesion_Local'] = stat.get('homeValue')
                elif name == 'Shots on target':
                    stats['Tiros_Arco_Local'] = stat.get('homeValue')

    except requests.exceptions.HTTPError as e:
        # Si da 403 Forbidden, significa que la API sigue bloqueada.
        pass 
    except Exception:
        pass
        
    return pd.Series(stats)

# --- EJECUCIÓN DEL BUCLE ---

if datos_partidos_base:
    df_base = pd.DataFrame(datos_partidos_base)
    
    print(f"Iniciando extracción de estadísticas avanzadas para {len(df_base)} partidos...")
    
    # Aplicar la función a la base de datos (se usa apply y se añade un sleep para cortesía)
    df_stats = df_base.apply(lambda row: obtener_estadisticas_avanzadas_sofascore(row), axis=1)
    
    # Unir la base y las estadísticas
    df_final_sofascore = pd.concat([df_base, df_stats], axis=1)

    print("\n" + "=" * 50)
    print(f"✅ Extracción Finalizada. Filas útiles: {len(df_final_sofascore)}")
    print("=" * 50)
    
    # Muestra y guarda el resultado
    display(df_final_sofascore[['ID_Partido', 'Posesion_Local', 'Tiros_Arco_Local', 'xG_Local']].head())
    
else:
    print("❌ Fallo crítico: No se pudieron recolectar los IDs de la página de resultados.")